[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **SportMap AI: Real-Time Homographic Projection for Tactical Field Analysis**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

## Overview

**HoloField** is an advanced computer vision pipeline designed to transform standard sports broadcast footage into a real-time, 2D tactical miniature. By combining **YOLO11 instance segmentation** with **Planar Homography**, the system projects dynamic entities from a distorted camera perspective onto a metric-accurate "digital twin" of the court.



The workflow covers a specialized end-to-end pipeline: from high-resolution retina mask extraction and perspective correction to real-time kinematic calculations. The result is a smooth, jitter-free "God-view" that tracks player positioning and velocity without the need for multiple camera angles or expensive wearable sensors.

#### Key Technical Features:
* **Deep Learning Inference:** Utilizing YOLO11x-Seg for pixel-perfect player and ball detection.
* **Perspective Transformation:** RANSAC-based Homography to map pixel coordinates to real-world meters.
* **Tactical Visualization:** Team-based color coding (Team A vs. Team B) and trajectory tail mapping.



#### Real-World Applications:
* **Professional Sports Analytics:** Automated tactical "God-view" generation for coaches and performance scouts.
* **Broadcasting & Fan Engagement:** Real-time AR overlays showing player speed and court coverage heatmaps.
* **Performance Monitoring:** Metric-accurate distance and velocity tracking for athletic training.
* **Digital Twin Synthesis:** Creating virtual environments for game replay and strategic simulation.
* **Automated Officiating:** Assisting in spatial positioning analysis and historical play review.

## Annotate your Custom dataset using Labellerr

 ***1. Visit the [Labellerr](https://www.labellerr.com/?utm_source=githubY&utm_medium=social&utm_campaign=github_clicks) website and click **“Sign Up”**.*** 

 ***2. After signing in, create your workspace by entering a unique name.***

 ***3. Navigate to your workspace’s API keys page (e.g., `https://<your-workspace>.labellerr.com/workspace/api-keys`) to generate your **API Key** and **API Secret**.***

 ***4. Store the credentials securely, and then use them to initialise the SDK or API client with `api_key`, `api_secret`.*** 

## Import Libraries

This section imports all the required libraries used throughout the project for computer vision, visualization, deep learning, and structured coding.


In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import torch
from typing import List, Tuple, Dict
from pathlib import Path

In [2]:
!git clone https://github.com/Labellerr/yolo_finetune_utils.git

fatal: destination path 'yolo_finetune_utils' already exists and is not an empty directory.


## Random Frame Extraction from Video

Extracts a fixed number of high-quality frames from one or more videos to create an image dataset for annotation and training.

### 🔹 Purpose
- Convert raw manufacturing videos into individual image frames  
- Perform random sampling to avoid frame bias  
- Prepare data for annotation and YOLO training  


In [3]:
from yolo_finetune_utils.frame_extractor import extract_random_frames

extract_random_frames(
    paths=['Untitled design.mp4'],
    total_images=50,
    out_dir="dataset_frames",
    jpg_quality=100,
    seed=42
)

[✓] Extracted 50 frames to folder: dataset_frames


## Download Annotations from Labellerr

After completing data labeling on the **Labellerr** platform, export the annotations in **COCO JSON format**.

Download the COCO JSON file from the Labellerr website and upload it into this project workspace to use it for further dataset preparation and training.

This COCO JSON file will be used in the next steps for:
- Frame–annotation alignment
- COCO → YOLO format conversion
- Model training and evaluation


# COCO to YOLO Format Conversion

Converts COCO-style segmentation annotations to YOLO segmentation dataset format.  
- Requires: `annotation.json` and images in `frames_output` directory.
- Output: Generated YOLO dataset folder.
- Parameters: allows train/val split, shuffling, and verbose mode.


In [3]:
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

coco_to_yolo_converter(
    json_path="export-#602ada0d-f6ae-4958-aaf7-845c0797c060.json",
    images_dir="dataset_frames",
    output_dir="yolo_dataset",
    use_split=True,
    train_ratio=0.7,
    val_ratio=0.2,
    test_ratio=0.1,
    shuffle=True,
    verbose=True
)


Conversion complete. Stats: {'train': 35, 'val': 10, 'test': 5}


{'stats': {'train': 35, 'val': 10, 'test': 5}, 'output_dir': 'yolo_dataset'}

# Load and Train YOLO Segmentation Model

Loads the YOLO segmentation model and trains it using the converted YOLO dataset.
- Data: Path to YOLO-style `data.yaml`
- Parameters: epochs, image size, batch size, device, dataloader workers, experiment name.


In [ ]:
from ultralytics import YOLO

# Load the nano segmentation model
model = YOLO('yolo11x-seg.pt')

# Start training
model.train(
    data='/content/holo/yolo_dataset/data.yaml',
    epochs=150,
    imgsz=640,
    device=0,
    project='/content/drive/MyDrive/holo',
    name='holo_final_run'
)

### 1. High-Resolution Instance Segmentation
This stage handles the core computer vision task: detecting and segmenting players using **YOLO11**. By enabling `retina_masks`, we ensure the green silhouettes are sharp and pixel-accurate, rather than blocky.

In [ ]:
from ultralytics import YOLO

# Load your trained model
model = YOLO('/content/drive/MyDrive/holo/holo_final_run3/weights/best.pt')

# Run tracking with High-Resolution (Retina) masks
results_generator = model.track(
    source='/content/drive/MyDrive/holo/Untitled design.mp4',
    conf=0.15,
    iou=0.5,
    persist=True,
    stream=True,         # Keeps RAM low
    retina_masks=True,   # CRITICAL: Forces high-resolution pixel masks
    boxes=False,         # Hides the bounding boxes
    save=True,           # Automatically saves the high-quality video
    project='/content/drive/MyDrive/holo',
    name='high_res_segmentation'
)

# Iterate to process
for result in results_generator:
    pass
   

### 2. High-Resolution Kinematic Tracking & Trajectories

This module focuses on the visual representation of player movement. It utilizes **YOLO11 Retina Masks** for sharp silhouettes and implements a custom trajectory system. By anchoring motion "tails" to the player's ground-contact point (the feet), the system provides a clean, professional aesthetic suitable for broadcast analysis.

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# 1. Load Model
model = YOLO('/content/drive/MyDrive/holo/holo_final_run3/weights/best.pt')

# 2. Paths
video_path = '/content/drive/MyDrive/holo/Untitled design.mp4'
output_path = '/content/drive/MyDrive/holo/clean_mask_trajectory.mp4'

# 3. Setup Video Properties
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# 4. Storage for Trajectories
trajectories = defaultdict(list)
mask_color = (0, 255, 0) # Green in BGR

# 5. Run Tracking
results_generator = model.track(
    source=video_path,
    conf=0.5,
    persist=True,
    stream=True,
    retina_masks=True, # High resolution
    boxes=False
)

for res in results_generator:
    frame = res.orig_img.copy()

    if res.masks is None or res.boxes.id is None:
        out.write(frame)
        continue

    track_ids = res.boxes.id.cpu().numpy().astype(int)
    class_ids = res.boxes.cls.cpu().numpy().astype(int)
    masks = res.masks.xy

    for i, (tid, cid) in enumerate(zip(track_ids, class_ids)):
        if cid == 1: # Only Players
            # --- 1. Draw Green Mask ---
            overlay = frame.copy()
            polygon = masks[i].astype(np.int32)
            cv2.fillPoly(overlay, [polygon], mask_color)
            cv2.addWeighted(overlay, 0.4, frame, 0.6, 0, frame)
            cv2.polylines(frame, [polygon], True, mask_color, 2)

            # --- 2. Update Trajectory (using feet) ---
            foot_point = polygon[np.argmax(polygon[:, 1])]
            cx, cy = int(foot_point[0]), int(foot_point[1])
            trajectories[tid].append((cx, cy))

            # Keep tail length manageable (30 frames)
            if len(trajectories[tid]) > 30:
                trajectories[tid].pop(0)

            # --- 3. Draw Trajectory Line ---
            if len(trajectories[tid]) >= 2:
                for j in range(1, len(trajectories[tid])):
                    cv2.line(frame, trajectories[tid][j-1], trajectories[tid][j], mask_color, 2)

            # --- 4. Track ID Drawing Removed ---
            # No text labels will be drawn on the frame

    out.write(frame)

cap.release()
out.release()
print(f"✅ Clean video saved to: {output_path}")

### 1. Interactive Geometric Calibration

The first phase of the pipeline involves establishing a spatial relationship between the video pixels and the real-world dimensions of the tennis court. This script provides an interactive GUI for manual coordinate marking, allowing the user to define key court intersections that serve as the anchor points for the **Homography Matrix**.

In [5]:
import cv2

points = []

def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Scale back the coordinates to original resolution if you scaled the frame
        print(f"Point Marked: {x}, {y}")
        points.append((x, y))
        cv2.circle(display_frame, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Mark Points", display_frame)

cap = cv2.VideoCapture('Untitled design.mp4')
ret, frame = cap.read()

if ret:
    display_frame = frame.copy()
    
    # 1. Create a resizable window
    cv2.namedWindow("Mark Points", cv2.WINDOW_NORMAL)
    
    # 2. (Optional) Resize the window to a viewable size (e.g., 1280x720)
    cv2.resizeWindow("Mark Points", 1280, 720)
    
    cv2.imshow("Mark Points", display_frame)
    cv2.setMouseCallback("Mark Points", click_event)

    print("Click points in order. Press 'q' to save and exit.")
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

Click points in order. Press 'q' to save and exit.
Point Marked: 531, 1689
Point Marked: 1230, 636
Point Marked: 2586, 639
Point Marked: 3306, 1692
Point Marked: 1008, 981
Point Marked: 1911, 981
Point Marked: 2814, 984


### 3. Digital Homography & Tactical Mapping

This core module acts as the "Brain" of the **HoloField** system. It translates raw pixel detections into a metric-accurate, top-down tactical miniature. By applying geometric transformation and temporal smoothing, it creates a professional-grade "God-view" of the match.



#### I. Mathematical Core: Planar Homography
The system establishes a relationship between the distorted camera view and a flat, metric representation of a tennis court ($10.97m \times 23.77m$). 

* **RANSAC Homography:** Using 7 calibrated points, the system calculates a $3 \times 3$ matrix ($H$) that maps any pixel $(x, y)$ to a real-world coordinate $(M_x, M_y)$. The use of RANSAC ensures that slight inaccuracies in manual point selection do not ruin the overall geometric projection.
* **Coordinate Buffering:** A $2.5m$ padding is added to all sides of the metric court, ensuring that players running behind the baseline or wide of the sidelines remain visible on the tactical map.

#### II. Kinematic Smoothing (EMA)
To eliminate "jitter" caused by pixel-level fluctuations in the YOLO segmentation masks, the system implements an **Exponential Moving Average (EMA)** filter:

$$Position_{smooth} = \alpha \cdot Position_{current} + (1 - \alpha) \cdot Position_{previous}$$

By setting $\alpha = 0.2$, the system prioritizes fluid motion over raw pixel noise, resulting in "broadcast-quality" movement of the player markers.

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# 1. SETUP HOMOGRAPHY (Keeping your verified points)
src_pts = np.array([[531, 1689], [1230, 636], [2586, 639], [3306, 1692],
                    [1008, 981], [1911, 981], [2814, 984]], dtype=np.float32)
pad = 2.5
dst_pts = np.array([[0+pad, 23.77+pad], [0+pad, 0+pad], [10.97+pad, 0+pad], [10.97+pad, 23.77+pad],
                    [0+pad, 11.885+pad], [5.485+pad, 11.885+pad], [10.97+pad, 11.885+pad]], dtype=np.float32)
H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# 2. MINIATURE SETUP
scale = 40
map_w, map_h = int((10.97 + 2*pad) * scale), int((23.77 + 2*pad) * scale)
net_line_y = 11.885

# --- SMOOTHING SETUP ---
# Dictionary to store the previous position of each player ID
# smoothed_pos = alpha * current + (1 - alpha) * previous
smoothed_positions = {}
alpha = 0.2  # Lower alpha = smoother movement, but more "lag". 0.1 to 0.3 is the sweet spot.

def get_miniature_base():
    court = np.zeros((map_h, map_w, 3), dtype=np.uint8)
    court[:] = (34, 139, 34)
    m_to_p = lambda x, y: (int((x + pad) * scale), int((y + pad) * scale))
    white = (255, 255, 255)
    cv2.rectangle(court, m_to_p(0, 0), m_to_p(10.97, 23.77), white, 2)
    for x_val in [1.37, 9.6]: cv2.line(court, m_to_p(x_val, 0), m_to_p(x_val, 23.77), white, 1)
    for y_val in [5.485, 18.285]: cv2.line(court, m_to_p(1.37, y_val), m_to_p(9.6, y_val), white, 1)
    cv2.line(court, m_to_p(5.485, 5.485), m_to_p(5.485, 18.285), white, 1)
    cv2.line(court, m_to_p(0, 11.885), m_to_p(10.97, 11.885), (0, 0, 0), 2)
    return court

# 3. STREAMING INFERENCE
model = YOLO('/content/drive/MyDrive/holo/holo_final_run3/weights/best.pt')
out = cv2.VideoWriter('smooth_holograph.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (map_w, map_h))

for result in model.track(source='/content/drive/MyDrive/holo/Untitled design.mp4', stream=True, persist=True, conf=0.15, retina_masks=True):
    mini_map = get_miniature_base()

    if result.masks is not None and result.boxes.id is not None:
        boxes = result.boxes
        ids = boxes.id.cpu().numpy().astype(int)
        clss = boxes.cls.cpu().numpy().astype(int)
        confs = boxes.conf.cpu().numpy()

        for i, obj_id in enumerate(ids):
            cls = clss[i]
            conf = confs[i]

            # REMOVED BALL: Only process players with high confidence
            if cls == 1 and conf >= 0.5:
                mask = result.masks.xy[i]
                foot_pixel = mask[np.argmax(mask[:, 1])]

                # Homography
                pt = np.array([foot_pixel[0], foot_pixel[1], 1.0]).reshape(3, 1)
                m_pt = np.dot(H, pt)
                mx, my = (m_pt[0]/m_pt[2]).item(), (m_pt[1]/m_pt[2]).item()

                # --- APPLY SMOOTHING ---
                if obj_id not in smoothed_positions:
                    smoothed_positions[obj_id] = (mx, my)
                else:
                    prev_x, prev_y = smoothed_positions[obj_id]
                    # EMA Formula
                    mx = alpha * mx + (1 - alpha) * prev_x
                    my = alpha * my + (1 - alpha) * prev_y
                    smoothed_positions[obj_id] = (mx, my)

                # --- DRAWING ---
                # Team A (Far): Red (0,0,255) | Team B (Near): Blue (255,0,0)
                dot_color = (0, 0, 255) if (my - pad) < net_line_y else (255, 0, 0)

                center = (int(mx * scale), int(my * scale))
                cv2.circle(mini_map, center, 15, dot_color, -1) # INCREASED SIZE to 15
                cv2.circle(mini_map, center, 15, (255, 255, 255), 2) # White border for contrast

    out.write(mini_map)

out.release()
print("Success! Smooth tracking video saved to smooth_holograph.mp4")

---

## 👨‍💻 About Labellerr's Hands-On Learning in Computer Vision

Thank you for exploring this **Labellerr Hands-On Computer Vision Cookbook**! We hope this notebook helped you learn, prototype, and accelerate your vision projects.  
Labellerr provides ready-to-run Jupyter/Colab notebooks for the latest models and real-world use cases in computer vision, AI agents, and data annotation.

---
## 🧑‍🔬 Check Our Popular Youtube Videos

Whether you're a beginner or a practitioner, our hands-on training videos are perfect for learning custom model building, computer vision techniques, and applied AI:

- [How to Fine-Tune YOLO on Custom Dataset](https://www.youtube.com/watch?v=pBLWOe01QXU)  
  Step-by-step guide to fine-tuning YOLO for real-world use—environment setup, annotation, training, validation, and inference.
- [Build a Real-Time Intrusion Detection System with YOLO](https://www.youtube.com/watch?v=kwQeokYDVcE)  
  Create an AI-powered system to detect intruders in real time using YOLO and computer vision.
- [Finding Athlete Speed Using YOLO](https://www.youtube.com/watch?v=txW0CQe_pw0)  
  Estimate real-time speed of athletes for sports analytics.
- [Object Counting Using AI](https://www.youtube.com/watch?v=smsjBBQcIUQ)  
  Learn dataset curation, annotation, and training for robust object counting AI applications.
---

## 🎦 Popular Labellerr YouTube Videos

Level up your skills and see video walkthroughs of these tools and notebooks on the  
[Labellerr YouTube Channel](https://www.youtube.com/@Labellerr/videos):

- [How I Fixed My Biggest Annotation Nightmare with Labellerr](https://www.youtube.com/watch?v=hlcFdiuz_HI) – Solving complex annotation for ML engineers.
- [Explore Your Dataset with Labellerr's AI](https://www.youtube.com/watch?v=LdbRXYWVyN0) – Auto-tagging, object counting, image descriptions, and dataset exploration.
- [Boost AI Image Annotation 10X with Labellerr's CLIP Mode](https://www.youtube.com/watch?v=pY_o4EvYMz8) – Refine annotations with precision using CLIP mode.
- [Boost Data Annotation Accuracy and Efficiency with Active Learning](https://www.youtube.com/watch?v=lAYu-ewIhTE) – Speed up your annotation workflow using Active Learning.

> 👉 **Subscribe** for Labellerr's deep learning, annotation, and AI tutorials, or watch videos directly alongside notebooks!

---

## 🤝 Stay Connected

- **Website:** [https://www.labellerr.com/](https://www.labellerr.com/)
- **Blog:** [https://www.labellerr.com/blog/](https://www.labellerr.com/blog/)
- **GitHub:** [Labellerr/Hands-On-Learning-in-Computer-Vision](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
- **LinkedIn:** [Labellerr](https://in.linkedin.com/company/labellerr)
- **Twitter/X:** [@Labellerr1](https://x.com/Labellerr1)

*Happy learning and building with Labellerr!*
